In [9]:
import altair as alt
import pandas as pd

source = pd.read_excel('OECD_Data.xlsx')

# Convert the units row (row 1) into a dictionary
units = source.iloc[0].to_dict()

# Create a dictionary for simplified names of the units
simplified_units = {
    "Percentage": "%",
    "Ratio": "ratio",
    "US Dollar": "$",
    "Average score": "average score",
    "Years": "years",
    "Micrograms per cubic metre": "µg/m³",
    "Hours": "hours"
}

# Remove the units row from the source data
source = source.iloc[1:]

# Reset the DataFrame index
source.reset_index(drop=True, inplace=True)

# Add units to each of the indicators
for column in source.columns:
    unit = units[column]
    if unit in simplified_units:
        source.rename(columns={column: f"{column} ({simplified_units[unit]})"}, inplace=True)


df = source

# Melt the DataFrame to have columns: 'Country', 'Indicator', 'Value'
df = df.melt(id_vars='Country', var_name='Indicator', value_name='Value')

# Filter data for the two indicators
dwelling_data = df[df['Indicator'] == 'Dwellings without basic facilities (%)']
leisure_data = df[df['Indicator'] == 'Time devoted to leisure and personal care (hours)']

# Merge the two dataframes on 'Country'
merged_data = pd.merge(dwelling_data, leisure_data, on='Country')

# Rename columns for clarity
merged_data.columns = ['Country', 'Indicator_x', 'Dwelling%', 'Indicator_y', 'LeisureTime']

# Create the scatter plot
scatter_plot = alt.Chart(merged_data).mark_circle(size=60).encode(
    x='Dwelling%',
    y='LeisureTime',
    tooltip=['Country', 'Dwelling%', 'LeisureTime']
).interactive()

scatter_plot


c:\Users\Dom\Dev\Newcastle\CSC3833\W1\venv\lib\site-packages\altair\utils\core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [ ]:
# Select numeric columns and ignore 'Country' column
numeric_cols = indicators

print(numeric_cols)

# Apply manual Min-Max normalization to the numeric columns
for col in numeric_cols:
    source[col] = (source[col] - source[col].min()) / (source[col].max() - source[col].min())

# Melt the DataFrame into a format where one row is one data point
source_melted = source.melt(id_vars='Country')

# Create the heatmap
heatmap = alt.Chart(source_melted).mark_rect().encode(
    x='Country:N',
    y='variable:N',
    color='value:Q',
    tooltip=[
        alt.Tooltip('Country:N', title='Country'),
        alt.Tooltip('variable:N', title='Indicator'),
        alt.Tooltip('value:Q', title='Value')
    ]
).properties(width=600, height=300)

heatmap